### Imports

In [ ]:
import requests
import json
import pandas as pd
import io
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import matplotlib.pyplot as plt

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Extracting entities

## Spacy & Transformers

In [ ]:
import spacy
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd

def extract_entities_multimodal(df, text_column):
  '''Extracts spacy and transformers entities.
  inputs: df : dataframe, text_column: name of the column to extract entities from
  '''
    # Load SpaCy model
    nlp_spacy = spacy.load("en_core_web_sm")

    # Set up the Transformers pipeline for NER with BERT
    tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
    model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
    nlp_transformers = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

    # Apply SpaCy NER
    df['spacy_entities'] = df[text_column].apply(lambda text: [(ent.text, ent.label_) for ent in nlp_spacy(text).ents])

    # Apply Transformers NER without filtering on 'B'
    df['transformers_entities'] = df[text_column].apply(
        lambda text: [(ent['entity_group'], ent['score'], " ".join(ent['word'])) for ent in nlp_transformers(text)]
    )

    return df

In [ ]:
papers_20_content_df = pd.read_csv(io.BytesIO(uploaded['entities_20_all_data (1).csv']))
papers_20_content_df['cleaned_content_text']

0     httpshalinraefrhal dec multidisciplinary open ...
1     httpshalsciencehal mar multidisciplinary open ...
2     people nature wileyonlinelibrarycomjournalpan ...
3     halshs httpsshshalsciencehalshs multidisciplin...
4     people nature wileyonlinelibrarycomjournalpan ...
5     httpshalinraefrhal apr multidisciplinary open ...
6     httpshalsciencehal jan multidisciplinary open ...
7     people nature wileyonlinelibrarycomjournalpan ...
8     people nature wileyonlinelibrarycomjournalpan ...
9     httpshalsciencehal jan multidisciplinary open ...
10    httpshalinraefrhal mar multidisciplinary open ...
11    people nature wileyonlinelibrarycomjournalpan ...
12    people nature wileyonlinelibrarycomjournalpan ...
13    httpsunivperphalsciencehal nov multidisciplina...
14    people nature wileyonlinelibrarycomjournalpan ...
15    httpshalsciencehal oct multidisciplinary open ...
16    httpshalsciencehal aug multidisciplinary open ...
17    httpshalsciencehal jul multidisciplinary o

In [ ]:
spacy_df = extract_entities_multimodal(papers_20_content_df, 'content_text')


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
spacy_df[['spacy_entitiesCounts','transformers_entitiesCounts']]

,spacy_entitiesCounts,transformers_entitiesCounts
0,"{('Schroth & McNeely', 'ORG'): 1, ('Franke', '...","{('ORG', 0.62263894, 'R e'): 1, ('ORG', 0.6063..."
1,"{('Driessen', 'PERSON'): 1, ('one', 'CARDINAL'...","{('LOC', 0.97415245, 'G u n'): 1, ('MISC', 0.5..."
2,"{('one', 'CARDINAL'): 6, ('2014', 'DATE'): 7, ...","{('ORG', 0.9442726, 'E x p e r i m e n t a l ..."
3,"{('one', 'CARDINAL'): 4, ('the Waraná Project'...","{('PER', 0.968828, 'F l o r e n c e F . P ..."
4,"{('2014', 'DATE'): 13, ('Košice', 'ORG'): 1, (...","{('ORG', 0.9642587, 'L e ó n'): 1, ('PER', 0.9..."
5,"{('https://doi.org/10.1007/s1162', 'PERSON'): ...","{('MISC', 0.9528946, '# # ç a i s'): 1, ('ORG'..."
6,"{('Hamer', 'PERSON'): 1, ('one', 'CARDINAL'): ...","{('ORG', 0.7539842, 'A n'): 1, ('LOC', 0.99960..."
7,"{('one', 'CARDINAL'): 5, ('85', 'DATE'): 1, ('...","{('LOC', 0.9274332, 'L i m a'): 1, ('ORG', 0.9..."
8,"{('the Creative Commons Attribution', 'ORG'): ...","{('PER', 0.9980516, 'E l e n a A n g u l o')..."
9,"{('K. J.,', 'PERSON'): 1, ('Haddaway', 'GPE'):...","{('PER', 0.8327225, 'T h o m a s'): 1, ('PER',..."


## Babel
(Davide's code)
http://localhost:8888/notebooks/OneDrive/Documentos/Thesis/kick-off/NamedEntitiesRecognition-Copy1.ipynb

In [ ]:
from google.colab import files
import pandas as pd
import io


uploaded = files.upload()

Saving entities_20_comparison-Counts_v5.xlsx to entities_20_comparison-Counts_v5 (2).xlsx


In [ ]:
# Assuming you have 'nerd' already imported and set up
papers_50taxref_df = pd.read_csv(io.BytesIO(uploaded['entities-50_TaxoREF-MD_v3.csv']))
papers_50taxref_df

,title_s,journalTitle_s,authLastName_s,authFullName_s,abstract_s,fileMain_s,ePublicationDateY_i,Complete,Genders,unknownGenders,content_text,cleaned_content_text,contentLen,TaxoRefEntitiesBB2,TaxoEntitiesMD2
0,"[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...","['Juliette Mariel', 'Stéphanie M. Carrière', '...","[""1. Interactions between farmers and agrobiod...",https://hal.inrae.fr/hal-03461601/document,NaN,True,"['female', 'female', 'male', 'male', 'unknown'...",33.333333,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,httpshalinraefrhal dec multidisciplinary open ...,48604,NaN,"[[('TAXREF:630768', 'Syzygium aromaticum', 1.0..."
1,['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...","['Matthew Mcdowell', 'Bruno David', 'Russell M...","[""Palaeontological animal bone deposits are ra...",https://hal.science/hal-03829927/document,NaN,True,"['male', 'male', 'male', 'female', 'unknown', ...",8.333333,HAL Id: hal-03829927 https://hal.science/hal-0...,httpshalsciencehal mar multidisciplinary open ...,47810,NaN,"[[('TAXREF:712345', 'Proto novaehollandiae', 0..."
2,['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']","['Kirsten Henderson', 'Michel Loreau']",['The dependence of humans on nature has come ...,https://hal.science/hal-02350603/document,NaN,True,"['female', 'male']",0.000000,People and Nature. 2019;1:31–43. | 31 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,47819,NaN,[]
3,"['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']","['Mélanie Congretel', 'Florence F. Pinton']","[""More than 30 years after the 1988 Declaratio...",https://shs.hal.science/halshs-02937670/document,NaN,True,"['female', 'female']",0.000000,HAL Id: halshs-02937670 https://shs.hal.scienc...,halshs httpsshshalsciencehalshs multidisciplin...,68504,NaN,"[[('TAXREF:841466', 'Paullinia cupana var. sor..."
4,['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Beaumelle', 'B...","['Werner Ulrich', 'Péter Batáry', 'Julia Baudr...",['Ample evidence suggests positive effects of ...,https://cnam.hal.science/hal-04080723/document,NaN,True,"['male', 'male', 'female', 'female', 'male', '...",0.000000,People and Nature. 2023;5:69–83. | 69 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,58880,NaN,"[[('TAXREF:210056', 'Cephalochilus Blüthgen, 1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1151,['Unidirectional response to bidirectional sel...,Ecology and Evolution,"['Renneville', 'Millot', 'Agostini', 'Carmigna...","['Clémentine Renneville', 'Alexis Millot', 'Si...",['Anthropogenic perturbations such as harvesti...,https://hal.sorbonne-universite.fr/hal-0295053...,NaN,True,"['female', 'male', 'male', 'male', 'female', '...",0.000000,HAL Id: hal-02950535 https://hal.sorbonne-univ...,httpshalsorbonneuniversitefrhal sep multidisci...,55877,NaN,NaN
1152,"['Think globally, measure locally: The MIREN s...",Ecology and Evolution,"['Haider', 'Lembrechts', 'Mcdougall', 'Pauchar...","['Sylvia Haider', 'Jonas J. Lembrechts', 'Keit...",['Climate change and other global change drive...,https://u-picardie.hal.science/hal-03619105/do...,NaN,True,"['female', 'male', 'male', 'male', 'male', 'fe...",13.793103,HAL Id: hal-03619105 https://u-picardie.hal.sc...,httpsupicardiehalsciencehal oct multidisciplin...,77580,NaN,NaN
1153,['Adaptation and plasticity in aboveground all...,Ecology and Evolution,"['Vizcaíno-Palomar', 'Ibáñez', 'González-Martí...","['Natalia Vizcaíno-Palomar', 'Inés Ibáñez', 'S...","[""Plant species aboveground allometry can be v...",https://hal.inrae.fr/hal-02636091/document,NaN,True,"['female', 'female', 'male', 'male', 'male']",0.000000,HAL Id: hal-02636091 https://hal.inrae.fr/hal-...,httpshalinraefrhal multidisciplinary open acce...,36497,NaN,NaN

In [ ]:
papers_50gbif_df = pd.read_csv(io.BytesIO(uploaded['entities-all-MD_GBIF_v3.csv']))
papers_50gbif_df


,fileMain_s,TaxoEntitiesMD2_GBIF
0,https://hal.inrae.fr/hal-03461601/document,"[[('GBIF:3183002', 'clove', 0.7634411454200745..."
1,https://hal.science/hal-03829927/document,"[[('GBIF:9065640', 'Tyto novaehollandiae novae..."
2,https://hal.science/hal-02350603/document,[]
3,https://shs.hal.science/halshs-02937670/document,"[[('GBIF:5786457', 'Sateré Marmoset', 0.771187..."
4,https://cnam.hal.science/hal-04080723/document,"[[('GBIF:4494312', 'Spilomena beata Blüthgen',..."
...,...,...
1151,https://hal.sorbonne-universite.fr/hal-0295053...,"[[('GBIF:2368377', 'Oryzias latipes', 1.0)], [..."
1152,https://u-picardie.hal.science/hal-03619105/do...,"[[('GBIF:4667268', 'Agustina', 0.7207658290863..."
1153,https://hal.inrae.fr/hal-02636091/document,"[[('GBIF:2438660', 'European Pine Vole', 0.730..."
1154,https://hal.science/hal-04167917/document,"[[('GBIF:2432958', 'Hypsignathus monstrosus', ..."


In [ ]:
papers_20_df = pd.read_csv(io.BytesIO(uploaded['entities-20_TaxoNERD_v2.csv']))
papers_20_df

,fileMain_s,content_text,TaxoEntitiesMD2,TaxoEntitiesMD,TaxoEntitiesBB,TaxoEntitiesBB2,TaxoRefEntitiesBB2
0,https://hal.inrae.fr/hal-03461601/document,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...","['clove agroforests', 'clove agroforests', 'cl...","['Stéphanie M. Carrière3', 'Verohanitra Rafidi...","[[('TAXREF:447477', 'Coffee tree', 0.891860187..."
1,https://hal.science/hal-03829927/document,HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['disc- faced owls', 'Tyto novaehollandae', 'M...","['disc- faced owls', 'Tyto novaehollandae', 'M...","[[('TAXREF:712345', 'Proto novaehollandiae', 0..."
2,https://hal.science/hal-02350603/document,People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...","['popu lation', 'lon ger', 'Muthukrishna', 'no...","['Muthukrishna', 'norfolk']","[[('TAXREF:592318', 'Coralliophila norfolk', 0..."
3,https://shs.hal.science/halshs-02937670/document,HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...","['guarana', 'guarana', 'Paullinia cupana', 'Ku...","['guarana', 'waraná', 'guarana', 'Paullinia cu...","[[('TAXREF:734243', 'Guarea guara', 0.78600811..."
4,https://cnam.hal.science/hal-04080723/document,People and Nature. 2023;5:69–83. | 69 wileyonl...,"['Péter Batáry2 | Julia', 'León', 'B. wA', 'Pé...","['Andrea erevkov Enrique', 'Blthgen', 'Lefchec...","['Serrano-', 'Carnero', 'Rembiakowska', 'Stahl...","['Léa Beaumelle', 'Beaumelle', 'Rembiałkowska'...",[]
5,https://hal.inrae.fr/hal-04085146/document,HAL Id: hal-04085146 https://hal.inrae.fr/hal-...,"['Katia Frangoudes, et', 'PERICLES project,1',...","['Katia Frangoudes, et al.', 'Larchive ouverte...","['Dimitra Mylona', 'Dimitra Mylona9|', 'Katia ...","['Dimitra Mylona', 'Katia Frangoudes', 'Dimitr...","[[('TAXREF:730671', 'Salmonia', 0.851587712764..."
6,https://hal.science/hal-03953883/document,HAL Id: hal-03953883 https://hal.science/hal-0...,"['Genève', 'Université', 'Aedes genus', 'Arctu...","['Larchive ouverte pluridisciplinaire HAL', 'A...","['Aedes', 'dengue', 'chikungunya', 'Aedes-', '...","['Aedes', 'dengue', 'chikungunya', 'yellow', '...","[[('TAXREF:188832', 'Aedes', 1.0), ('TAXREF:82..."
7,https://hal.science/hal-03482462/document,People and Nature. 2022;4:351–364. | 351 wiley...,"['Llañocancha', 'palm trees', 'CEDES', 'Houria...","['Llaocancha', 'palm trees', 'CEDES', 'Houria ...","['Ysica Quispe Conde3,6', 'Houria Djoudi7', 'B...","['Yésica Quispe Conde3,6', 'Houria Djoudi7', '...","[[('TAXREF:185961', 'birds', 0.999999880790710..."
8,https://hal.inrae.fr/hal-04285321/document,People and Nature. 2023;5:1561–1576. | 1561 wi...,"['starlings', 'Sturnus vulgaris', 'Alien Taxa ...","['Environment (DFFE)', 'starlings', 'Sturnus v...","['common starlings', 'Sturnus vulgaris', 'Alie...","['common starlings', 'Sturnus vulgaris', 'Alie...","[[('TAXREF:4516', 'Common Starling', 0.7818087..."
9,https://hal.science/hal-03511302/document,HAL Id: hal-03511302 https://hal.science/hal-0...,"['CNRS- INEE', 'Carmenta et', 'procedural', 'a...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['whakapapa', 'tuakana-', 'teina', 'Alaska sal...","['whakapapa', 'tuakana-', 'teina', 'Alaska sal...","[[('TAXREF:542538', 'Olive Anthias', 0.7270555..."


In [ ]:
import pandas as pd

# Assuming you have four dataframes: papers_20_df, papers_50taxref_df, papers_50gbif_df, papers_20_content_df

# First, merge the first two datasets on 'fileMain_s'
merged_df = pd.merge(papers_20_df, papers_50taxref_df, on='fileMain_s', how='inner')

# Now merge the result with the third dataset
merged_df = pd.merge(merged_df, papers_50gbif_df, on='fileMain_s', how='inner')

# Finally, merge the result with the fourth dataset
merged_df = pd.merge(merged_df, papers_20_content_df, on='fileMain_s', how='inner')

# Now merged_df contains the data combined from all four dataframes where 'fileMain_s' matches across all of them
merged_df

,fileMain_s,content_text_x,TaxoEntitiesMD2_x,TaxoEntitiesMD_x,TaxoEntitiesBB,TaxoEntitiesBB2,TaxoRefEntitiesBB2_x,title_s,journalTitle_s,authLastName_s,...,unknownGenders,content_text_y,cleaned_content_text,contentLen,TaxoRefEntitiesBB2_y,TaxoEntitiesMD2_y,TaxoEntitiesMD2_GBIF,content_text,TaxoEntitiesMD2,TaxoEntitiesMD_y
0,https://hal.inrae.fr/hal-03461601/document,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...","['clove agroforests', 'clove agroforests', 'cl...","['Stéphanie M. Carrière3', 'Verohanitra Rafidi...","[[('TAXREF:447477', 'Coffee tree', 0.891860187...","[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...",...,33.333333,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,httpshalinraefrhal dec multidisciplinary open ...,48604,NaN,"[[('TAXREF:630768', 'Syzygium aromaticum', 1.0...","[[('GBIF:3183002', 'clove', 0.7634411454200745...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',..."
1,https://hal.science/hal-03829927/document,HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['disc- faced owls', 'Tyto novaehollandae', 'M...","['disc- faced owls', 'Tyto novaehollandae', 'M...","[[('TAXREF:712345', 'Proto novaehollandiae', 0...",['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...",...,8.333333,HAL Id: hal-03829927 https://hal.science/hal-0...,httpshalsciencehal mar multidisciplinary open ...,47810,NaN,"[[('TAXREF:712345', 'Proto novaehollandiae', 0...","[[('GBIF:9065640', 'Tyto novaehollandiae novae...",HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C..."
2,https://hal.science/hal-02350603/document,People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...","['popu lation', 'lon ger', 'Muthukrishna', 'no...","['Muthukrishna', 'norfolk']","[[('TAXREF:592318', 'Coralliophila norfolk', 0...",['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']",...,0.000000,People and Nature. 2019;1:31–43. | 31 wileyonl...,people nature wileyonlinelibrarycomjournalpan ...,47819,NaN,[],[],People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',..."
3,https://shs.hal.science/halshs-02937670/document,HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...","['guarana', 'guarana', 'Paullinia cupana', 'Ku...","['guarana', 'waraná', 'guarana', 'Paullinia cu...","[[('TAXREF:734243', 'Guarea guara', 0.78600811...","['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']",...,0.000000,HAL Id: halshs-02937670 https://shs.hal.scienc...,halshs httpsshshalsciencehalshs multidisciplin...,68504,NaN,"[[('TAXREF:841466', 'Paullinia cupana var. sor...","[[('GBIF:5786457', 'Sateré Marmoset', 0.771187...",HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair..."
4,https://cnam.hal.science/hal-04080723/document,People and Nature. 2023;5:69–83. | 69 wileyonl...,"['Péter Batáry2 | Julia', 'León', 'B. wA', 'Pé...","['Andrea erevkov Enrique', 'Blthgen', 'Lefchec...","['Serrano-', 'Carnero', 'Rembiakowska', 'Stahl...","['Léa Beaumelle', 'Beaumelle', 'Rembiałkowska'...",[],['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Bea

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import pandas as pd



# Load the tokenizer and model outside the function to avoid reloading them each time the function is called
tokenizer = AutoTokenizer.from_pretrained("Babelscape/wikineural-multilingual-ner")
model = AutoModelForTokenClassification.from_pretrained("Babelscape/wikineural-multilingual-ner")
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

def extract_entities(df, text_column):
    # Initialize the new column for entities
    df['BABEL_entities'] = None  # Ensure this column exists

    # Define the maximum chunk size
    max_length = tokenizer.model_max_length

    # Temporary storage for entity lists
    entity_lists = []

    # Loop over each row in the dataframe
    for text in df[text_column]:
        entity_list = []
        chunks = [text[i:i+max_length] for i in range(0, len(text), max_length)]

        for chunk in chunks:
            results = nlp(chunk)
            entities = [f"{result['word']} ({result['entity_group']})" for result in results]
            entity_list.extend(entities)

        entity_lists.append(entity_list)

    # Assign all entities at once
    df['BABEL_entities'] = entity_lists

    return df

# Example usage
papers_entities_df = extract_entities(merged_df, 'content_text')
papers_entities_df

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/token_classification.py:168: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="simple"` instead.
  warnings.warn(


,fileMain_s,content_text_x,TaxoEntitiesMD2_x,TaxoEntitiesMD_x,TaxoEntitiesBB,TaxoEntitiesBB2,TaxoRefEntitiesBB2_x,title_s,journalTitle_s,authLastName_s,...,cleaned_content_text,contentLen,TaxoRefEntitiesBB2_y,TaxoEntitiesMD2_y,TaxoEntitiesMD2_GBIF,content_text,TaxoEntitiesMD2,TaxoEntitiesMD_y,entities,BABEL_entities
0,https://hal.inrae.fr/hal-03461601/document,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...","['clove agroforests', 'clove agroforests', 'cl...","['Stéphanie M. Carrière3', 'Verohanitra Rafidi...","[[('TAXREF:447477', 'Coffee tree', 0.891860187...","[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...",...,httpshalinraefrhal dec multidisciplinary open ...,48604,NaN,"[[('TAXREF:630768', 'Syzygium aromaticum', 1.0...","[[('GBIF:3183002', 'clove', 0.7634411454200745...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...",None,"[France (LOC), Creative Commons Attribution (M..."
1,https://hal.science/hal-03829927/document,HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['disc- faced owls', 'Tyto novaehollandae', 'M...","['disc- faced owls', 'Tyto novaehollandae', 'M...","[[('TAXREF:712345', 'Proto novaehollandiae', 0...",['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...",...,httpshalsciencehal mar multidisciplinary open ...,47810,NaN,"[[('TAXREF:712345', 'Proto novaehollandiae', 0...","[[('GBIF:9065640', 'Tyto novaehollandiae novae...",HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...",None,"[France (LOC), Cloggs Cave (LOC), SE (LOC), Au..."
2,https://hal.science/hal-02350603/document,People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...","['popu lation', 'lon ger', 'Muthukrishna', 'no...","['Muthukrishna', 'norfolk']","[[('TAXREF:592318', 'Coralliophila norfolk', 0...",['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']",...,people nature wileyonlinelibrarycomjournalpan ...,47819,NaN,[],[],People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...",None,"[People and Nature (MISC), Biraben (PER), Rose..."
3,https://shs.hal.science/halshs-02937670/document,HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...","['guarana', 'guarana', 'Paullinia cupana', 'Ku...","['guarana', 'waraná', 'guarana', 'Paullinia cu...","[[('TAXREF:734243', 'Guarea guara', 0.78600811...","['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']",...,halshs httpsshshalsciencehalshs multidisciplin...,68504,NaN,"[[('TAXREF:841466', 'Paullinia cupana var. sor...","[[('GBIF:5786457', 'Sateré Marmoset', 0.771187...",HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...",None,"[France (LOC), Mélanie Congretel (PER), Floren..."
4,https://cnam.hal.science/hal-04080723/document,People and Nature. 2023;5:69–83. | 69 wileyonl...,"['Péter Batáry2 | Julia', 'León', 'B. wA', 'Pé...","['Andrea erevkov Enrique', 'Blthgen', 'Lefchec...","['Serrano-', 'Carnero', 'Rembiakowska', 'Stahl...","['Léa Beaumelle', 'Beaumelle', 'Rembiałkowska'...",[],['From biodiversity to health: Quantifying the...,People and Nature,"['Ulrich', 'Batáry', 'Baudry', 'Beaumelle', 'B..."

## Auxiliar functions

In [ ]:
# Assuming papers_20_all_data_df is the DataFrame resulting from the merge that has duplicate columns with suffixes _x and _y.
# Sample DataFrame initialization - replace this with your actual DataFrame
# papers_20_all_data_df = your_actual_dataframe_here

# Function to rename and drop duplicated columns
def drop_duplicate_columns(df):
    # Get a list of columns that end with '_x' or '_y'
    duplicated_columns = [col for col in df.columns if col.endswith('_x') or col.endswith('_y')]

    # For each pair of duplicated columns, keep the one ending with '_x' and drop the one with '_y'
    for col in duplicated_columns:
        if col.endswith('_x'):
            # Extract the base column name without '_x'
            base_col_name = col[:-2]
            # Check if the corresponding '_y' column exists
            col_y = base_col_name + '_y'
            if col_y in df.columns:
                # Keep the '_x' column and rename it to the base name
                df.rename(columns={col: base_col_name}, inplace=True)
                # Drop the '_y' column
                df.drop(columns=[col_y], inplace=True)

    return df

# Apply the function to the DataFrame
papers_20_all_data_df = drop_duplicate_columns(papers_20_all_data_df)
papers_20_all_data_df.head()  # Display the first few rows of the cleaned DataFrame


,fileMain_s,content_text,TaxoEntitiesMD2,TaxoEntitiesMD,TaxoEntitiesBB,TaxoEntitiesBB2,TaxoRefEntitiesBB2,title_s,journalTitle_s,authLastName_s,...,contentLen,TaxoEntitiesMD2_GBIF,content_text,TaxoEntitiesMD2,entities,BABEL_entities,spacy_entities,transformers_entities,spacy_entitiesCounts,transformers_entitiesCounts
0,https://hal.inrae.fr/hal-03461601/document,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...","['clove agroforests', 'clove agroforests', 'cl...","['Stéphanie M. Carrière3', 'Verohanitra Rafidi...","[[('TAXREF:447477', 'Coffee tree', 0.891860187...","[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...",...,48604,"[[('GBIF:3183002', 'clove', 0.7634411454200745...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...",NaN,"['France (LOC)', 'Creative Commons Attribution...","[('1', 'CARDINAL'), ('2021', 'DATE'), ('HAL', ...","[('ORG', 0.940115, 'H A L'), ('ORG', 0.9775882...","{('Schroth & McNeely', 'ORG'): 1, ('Franke', '...","{('ORG', 0.62263894, 'R e'): 1, ('ORG', 0.6063..."
1,https://hal.science/hal-03829927/document,HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['disc- faced owls', 'Tyto novaehollandae', 'M...","['disc- faced owls', 'Tyto novaehollandae', 'M...","[[('TAXREF:712345', 'Proto novaehollandiae', 0...",['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...",...,47810,"[[('GBIF:9065640', 'Tyto novaehollandiae novae...",HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...",NaN,"['France (LOC)', 'Cloggs Cave (LOC)', 'SE (LOC...","[('9 Mar 2024', 'DATE'), ('HAL', 'ORG'), ('Fra...","[('ORG', 0.94845426, 'H A L'), ('ORG', 0.98357...","{('Driessen', 'PERSON'): 1, ('one', 'CARDINAL'...","{('LOC', 0.97415245, 'G u n'): 1, ('MISC', 0.5..."
2,https://hal.science/hal-02350603/document,People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...","['popu lation', 'lon ger', 'Muthukrishna', 'no...","['Muthukrishna', 'norfolk']","[[('TAXREF:592318', 'Coralliophila norfolk', 0...",['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']",...,47819,[],People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...",NaN,"['People and Nature (MISC)', 'Biraben (PER)', ...","[('2019;1:31–43', 'CARDINAL'), ('31', 'CARDINA...","[('ORG', 0.8812945, 'P e o p l e a n d N a...","{('one', 'CARDINAL'): 6, ('2014', 'DATE'): 7, ...","{('ORG', 0.9442726, 'E x p e r i m e n t a l ..."
3,https://shs.hal.science/halshs-02937670/document,HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...","['guarana', 'guarana', 'Paullinia cupana', 'Ku...","['guarana', 'waraná', 'guarana', 'Paullinia cu...","[[('TAXREF:734243', 'Guarea guara', 0.78600811...","['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']",...,68504,"[[('GBIF:5786457', 'Sateré Marmoset', 0.771187...",HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...",NaN,"['France (LOC)', 'Mélanie Congretel (PER)', 'F...","[('11 May 2022', 'DATE'), ('HAL', 'ORG'), ('Fr...","[('ORG', 0.9361925, 'H A L'), ('ORG', 0.985233...","{('one', 'CARDINAL'): 4, ('the Waraná Project'...","{('PER', 0.968828, 'F l o r e n c e F . P ..."
4,https://cnam.hal.science/hal-04080723/document,People and Nature. 2023;5:69–83. | 69 wileyonl...,"['Péter Batáry2 | Julia', 'León', 'B. wA', 'Pé...","['Andrea erevkov Enrique', 'Blthgen', 'L

In [ ]:
import pandas as pd
import ast

def count_unique_entities_per_row_and_sort(column):
    # Process each row as a list of tuples
    return column.apply(lambda x: {i: x.count(i) for i in set(x)} if isinstance(x, list) and x else {})

def safely_convert_to_list(column):
    # Convert strings that represent lists or tuples safely
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and (x.startswith('[') or x.startswith('(')) else x)

# Assuming papers_df is defined and preprocessed correctly
entities_df = papers_df.copy()

columns_to_process = [
    'TaxoEntitiesMD2', 'TaxoEntitiesMD', 'TaxoEntitiesBB2',
    'TaxoEntitiesBB', 'TaxoRefEntitiesBB2_SingleList',
    'TaxoRefEntitiesBB2_x', 'TaxoEntitiesMD2_y', 'TaxoEntitiesMD2_GBIF'
]

for col in columns_to_process:
    try:
        # Safely convert columns to lists or tuples if they are not already
        entities_df[col] = safely_convert_to_list(entities_df[col])

        # Apply the function and store the result in a new column
        new_col_name = f'{col}Counts'
        entities_df[new_col_name] = count_unique_entities_per_row_and_sort(entities_df[col])
    except Exception as e:
        print(f"Error processing column {col}: {e}")

# Display the DataFrame with the new columns
entities_df


In [ ]:
# Define the list of columns you want to keep
columns_to_keep = [
    'TaxoEntitiesMD',
    'TaxoEntitiesBB',
    'TaxoEntitiesMD2',
    'TaxoEntitiesBB2',
    'TaxoRefEntitiesBB2',
    'TaxoEntitiesMD2_y',
    'TaxoEntitiesMD2_GBIF',
    'BABEL',
    'Spacy',
    'Transformers'
]

# Filter the DataFrame to keep only these columns
filtered_papers = papers_20_all_data_df[columns_to_keep]
# Sort the column names alphabetically
sorted_columns = sorted(filtered_papers.columns)

# Reorder the DataFrame using the sorted list of column names
filtered_papers = filtered_papers[sorted_columns]

filtered_papers

KeyError: "['TaxoEntitiesMD', 'TaxoEntitiesBB', 'TaxoEntitiesMD2', 'TaxoEntitiesBB2', 'TaxoRefEntitiesBB2', 'TaxoEntitiesMD2_y', 'TaxoEntitiesMD2_GBIF'] not in index"

In [ ]:
filtered_papers.to_csv('entities_comparison.csv', index=False, encoding='utf-8')
filtered_papers.to_excel('entities_comparison.xlsx', index=False, engine='openpyxl')


from google.colab import files
files.download('entities_comparison.csv')
files.download('entities_comparison.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import ast

entities_df = merged_df.copy()

def count_unique_entities_per_row_and_sort(column):
    # Process each row as a list of tuples
    return column.apply(lambda x: {i: x.count(i) for i in set(x)} if isinstance(x, list) and x else {})

def safely_convert_to_list(column):
    # Convert strings that represent lists or tuples safely
    return column.apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and (x.startswith('[') or x.startswith('(')) else x)


columns_to_process = [
    'spacy_entities',
    'transformers_entities'
]

for col in columns_to_process:
    try:
        # Safely convert columns to lists or tuples if they are not already
        entities_df[col] = safely_convert_to_list(entities_df[col])

        # Apply the function and store the result in a new column
        new_col_name = f'{col}Counts'
        entities_df[new_col_name] = count_unique_entities_per_row_and_sort(entities_df[col])
    except Exception as e:
        print(f"Error processing column {col}: {e}")

# Display the DataFrame with the new columns
entities_df

,fileMain_s,content_text_x,TaxoEntitiesMD2_x,TaxoEntitiesMD_x,TaxoEntitiesBB,TaxoEntitiesBB2,TaxoRefEntitiesBB2_x,title_s,journalTitle_s,authLastName_s,...,TaxoEntitiesMD2_GBIF,content_text,TaxoEntitiesMD2,TaxoEntitiesMD_y,entities,BABEL_entities,spacy_entities,transformers_entities,spacy_entitiesCounts,transformers_entitiesCounts
0,https://hal.inrae.fr/hal-03461601/document,HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...","['clove agroforests', 'clove agroforests', 'cl...","['Stéphanie M. Carrière3', 'Verohanitra Rafidi...","[[('TAXREF:447477', 'Coffee tree', 0.891860187...","[""Exploring farmers' agrobiodiversity manageme...",People and Nature,"['Mariel', 'Carrière', 'Penot', 'Danthu', 'Raf...",...,"[[('GBIF:3183002', 'clove', 0.7634411454200745...",HAL Id: hal-03461601 https://hal.inrae.fr/hal-...,"['M. Carrière, Eric A. Penot', 'Verohanitra Ra...","['Verohanitra Rafidison,', 'Vanesse Labeyrie',...",None,"[France (LOC), Creative Commons Attribution (M...","[(1, CARDINAL), (2021, DATE), (HAL, ORG), (Fra...","[(ORG, 0.940115, H A L), (ORG, 0.9775882, H A ...","{('Schroth & McNeely', 'ORG'): 1, ('Franke', '...","{('ORG', 0.62263894, 'R e'): 1, ('ORG', 0.6063..."
1,https://hal.science/hal-03829927/document,HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...","['disc- faced owls', 'Tyto novaehollandae', 'M...","['disc- faced owls', 'Tyto novaehollandae', 'M...","[[('TAXREF:712345', 'Proto novaehollandiae', 0...",['Interpreting the mammal deposits of Cloggs C...,People and Nature,"['Mcdowell', 'David', 'Mullett', 'Fresløv', 'D...",...,"[[('GBIF:9065640', 'Tyto novaehollandiae novae...",HAL Id: hal-03829927 https://hal.science/hal-0...,"['GunaiKurnai Land and Waters', 'Cloggs Cave',...","['Larchive ouverte pluridisciplinaire HAL', 'C...",None,"[France (LOC), Cloggs Cave (LOC), SE (LOC), Au...","[(9 Mar 2024, DATE), (HAL, ORG), (France, GPE)...","[(ORG, 0.94845426, H A L), (ORG, 0.9835798, H ...","{('Driessen', 'PERSON'): 1, ('one', 'CARDINAL'...","{('LOC', 0.97415245, 'G u n'): 1, ('MISC', 0.5..."
2,https://hal.science/hal-02350603/document,People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...","['popu lation', 'lon ger', 'Muthukrishna', 'no...","['Muthukrishna', 'norfolk']","[[('TAXREF:592318', 'Coralliophila norfolk', 0...",['An ecological theory of changing human popul...,People and Nature,"['Henderson', 'Loreau']",...,[],People and Nature. 2019;1:31–43. | 31 wileyonl...,"['anti‐human’', 'Anderies', 'Malthus’ theory',...","['EXISTINGTHEORIESANDMODELS', 'medieval agri',...",None,"[People and Nature (MISC), Biraben (PER), Rose...","[(2019;1:31–43, CARDINAL), (31, CARDINAL), (Bi...","[(ORG, 0.8812945, P e o p l e a n d N a t ...","{('one', 'CARDINAL'): 6, ('2014', 'DATE'): 7, ...","{('ORG', 0.9442726, 'E x p e r i m e n t a l ..."
3,https://shs.hal.science/halshs-02937670/document,HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...","['guarana', 'guarana', 'Paullinia cupana', 'Ku...","['guarana', 'waraná', 'guarana', 'Paullinia cu...","[[('TAXREF:734243', 'Guarea guara', 0.78600811...","['Local knowledge, know‐how and knowledge mobi...",People and Nature,"['Congretel', 'Pinton']",...,"[[('GBIF:5786457', 'Sateré Marmoset', 0.771187...",HAL Id: halshs-02937670 https://shs.hal.scienc...,"['Sateré', 'Paullinia cupana (Kunth var. sorbi...","['Mlanie', 'Larchive ouverte pluridisciplinair...",None,"[France (LOC), Mélanie Congretel (PER), Floren...","[(11 May 2022, DATE), (HAL, ORG), (France, GPE...","[(ORG, 0.9361925, H A L), (ORG, 0.98523307, H ...","{('one', 'CARDINAL'): 4, ('the Waraná Project'...","{('PER', 0.968828, 'F l o r e n c e F